<div style="padding: 10px; margin-bottom: 10px; border: 1px solid #333333;">

## Support Mission 6 ［ラベル付け画像に基づく機械学習モデルの訓練］
- このSupport Missionでは集めた画像をもとにJetBotを動かすためのモデルを作成します．
    
#### 完了の定義
- Support Missionディレクトリ内にbest_model.pthファイルがある．

## <span style="color: red; ">NEW</span> タグ：[ラベル付けに基づく機械学習モデルの訓練]

In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

#### データセットインスタンスの作成
- 集めた画像データを正規化します．

In [2]:
dataset = datasets.ImageFolder(
    'dataset',
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

#### データの分割
- データセットをトレーニング用セットと精度のテスト用セットに分割します．

In [3]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - 50, 50])

#### DataLoaderインスタンスの生成
- ２つのDataLoaderインスタンスを生成します．
- これらのインスタンスはデータのシャッフルやミニバッチの生成などを行います．

In [4]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

#### ニューラルネットワークを定義する
- トレーニングするニューラルネットワークを定義します．
- torchvisionパッケージはすぐに使用可能な事前トレーニング済みモデルを提供してくれます．
- 転送学習と呼ばれるプロセスでは，利用可能なデータがはるかに少ない可能性がある新しいタスクのために<br>
  事前にトレーニングされた（数百万の画像でトレーニングされた）モデルを再利用できます．
- 事前トレーニング済みモデルの元のトレーニングで学習した重要な機能は，新しいタスクに再利用できます．
- 今回はalexnetモデルを使用します．

In [5]:
model = models.alexnet(pretrained=True)

#### <span style="color: red; ">重要！</span>
- 元々，alexnetモデルはクラスラベルが1000個あるデータセット用にトレーニングされているが，<br>
  今回のSupport Missionのデータセットにはクラスラベルがfreeとblockedの２つしかありません．
- なので，最終レイヤーを出力層が２つしかない新しいレイヤーに置き換えます．

In [6]:
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

- GPUで実行するためのにモデルを転送します．

In [7]:
device = torch.device('cuda')
model = model.to(device)

#### トレーニング
- 以下のコードを使用して30エポックのニューラルネットワークをトレーニングし，<br>
  各エポックの後に最高のパフォーマンスモデルを保存します．
- エポックとは，データを完全に把握することです．
- このセルは他のセルよりも実行に時間がかかるのですぐに再起動しないようにしてください．

In [8]:
NUM_EPOCHS = 30
BEST_MODEL_PATH = 'best_model.pth'
best_accuracy = 0.0

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(NUM_EPOCHS):
    
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
    
    test_error_count = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
    
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))
    print('%d: %f' % (epoch, test_accuracy))
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_accuracy = test_accuracy

0: 0.680000
1: 1.000000
2: 1.000000
3: 1.000000
4: 0.980000
5: 0.980000
6: 0.980000
7: 1.000000
8: 0.980000
9: 0.980000
10: 1.000000
11: 1.000000
12: 1.000000
13: 1.000000
14: 1.000000
15: 1.000000
16: 1.000000
17: 1.000000
18: 1.000000
19: 1.000000
20: 1.000000
21: 1.000000
22: 1.000000
23: 1.000000
24: 1.000000
25: 1.000000
26: 1.000000
27: 1.000000
28: 1.000000
29: 1.000000
